# EMPLOYEE ATTRITION ANALYSIS

One of the most critical challenges in any business is the retention of employees for profitability and productivity. However, it is very difficult to control the attrition rate. Attrition affects the company's productivity by increasing the workload of the remaining employees and hiring and training new employees. In order to retain employees in the company, it is crucial to understand what variables factor into attrition.

In this analysis, I will explore employee attrition, understand the factors that attribute to attrition, and build a model based on my analysis. This will help me understand why employees left this company.

This report will be broken down into six sections.
- **Part 1: Load Libraries & Explore Data**
    - I will install and load in the appropriate libraries needed to analyze the dataset.
- **Part 2: Plot the Dataset**
    - I will further explore the data, using graphs that will help me to visually understand the effects of certain variables. I will analyze my graphs to answer some questions presented by management.
- **Part 3: Prepare the Dataset for Modeling**
    - After exploring the data, I will prepare the dataset for modeling by assuring the dataset is complete and has no missing values.  
- **Part 4: Create & Evaluate Prediction Models**
    - In this step, I will create prediction models using two methods – logistic regression and classification. 
- **Part 5: Predict & Test Attrition using Model**
    - After creating my prediction models, I will use my model to test its accuracy on predicting attrition.
- **Part 6: Save Predictions File & Recommendations**
    - Lastly, I will save my predictions into a new file and recommend suggestions based on my analysis of my model.


# PART 1: LOAD LIBRARIES & EXPLORE DATA

In [1]:
# Load the appropriate libraries
shh <- suppressPackageStartupMessages

shh(install.packages('dplyr'))
shh(install.packages('randomForest'))
shh(install.packages('pROC'))

shh(library(ggplot2))
shh(library(caret))
shh(library(dplyr))
shh(library(randomForest))
shh(library(pROC))

Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)
Installing package into ‘/home/nbuser/R’
(as ‘lib’ is unspecified)


In [2]:
# Load the CSV file with employee data
hr_data <- read.csv("http://bit.ly/hr_data_project")

## Explore the dataset
After loading in the appropriate libraries and reading in the CSV file, I explored the employee data so I could get familiar with what I need to evaluate. I was able to get a preview of what the data looked like, the type of each variable, and some statistics of each variable in the dataset.

In [3]:
# Preview the first few records in the dataset
head(hr_data)

Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6


In [4]:
# List and describe each of the fields in the dataset
str(hr_data)

'data.frame':	1470 obs. of  25 variables:
 $ Attrition              : int  1 0 1 0 0 0 0 0 0 0 ...
 $ EmployeeNumber         : int  1 2 4 5 7 8 10 11 12 13 ...
 $ Age                    : int  41 49 37 33 27 32 59 30 38 36 ...
 $ Gender                 : Factor w/ 2 levels "Female","Male": 1 2 2 1 2 2 1 2 2 2 ...
 $ MaritalStatus          : Factor w/ 3 levels "Divorced","Married",..: 3 2 3 2 2 3 2 1 3 2 ...
 $ Department             : Factor w/ 3 levels "Human Resources",..: 3 2 2 2 2 2 2 2 2 2 ...
 $ JobRole                : Factor w/ 9 levels "Healthcare Representative",..: 8 7 3 7 3 3 3 3 5 1 ...
 $ MonthlyIncome          : int  5993 5130 2090 2909 3468 3068 2670 2693 9526 5237 ...
 $ OverTime               : Factor w/ 2 levels "No","Yes": 2 1 2 2 1 1 2 1 1 1 ...
 $ EducationField         : Factor w/ 6 levels "Human Resources",..: 2 2 5 2 4 2 4 2 2 4 ...
 $ BusinessTravel         : Factor w/ 3 levels "Non-Travel","Travel_Frequently",..: 3 2 3 2 3 2 3 3 2 3 ...
 $ DistanceFromHome   

In [5]:
# Calculate descriptive stats for each of the fields in the dataset
summary(hr_data)

   Attrition      EmployeeNumber        Age           Gender     MaritalStatus
 Min.   :0.0000   Min.   :   1.0   Min.   :18.00   Female:588   Divorced:327  
 1st Qu.:0.0000   1st Qu.: 491.2   1st Qu.:30.00   Male  :882   Married :673  
 Median :0.0000   Median :1020.5   Median :36.00                Single  :470  
 Mean   :0.1612   Mean   :1024.9   Mean   :36.92                              
 3rd Qu.:0.0000   3rd Qu.:1555.8   3rd Qu.:43.00                              
 Max.   :1.0000   Max.   :2068.0   Max.   :60.00                              
                                                                              
                  Department                       JobRole    MonthlyIncome  
 Human Resources       : 63   Sales Executive          :326   Min.   : 1009  
 Research & Development:961   Research Scientist       :292   1st Qu.: 2911  
 Sales                 :446   Laboratory Technician    :259   Median : 4919  
                              Manufacturing Director   :

# PART 2: PLOT THE DATASET

Now that I'm familiar with the data, I created plots and graphs of the data using the ggplot2 function. This helped me gain a better understanding by visually capturing the data and answering some questions asked by management.

### QUESTION 1: Does one gender tend to quit more frequently than the other?

Based on the first chart, it looks like more males left the company. Although it looks this way, you have to also notice that there are a lot more male employees than female employees, so it makes sense that there will be more males leaving. The second chart shows the attrition percentage of female and male employees. This chart shows that the percentages of each gender leaving are relatively close. Therefore, one gender does not tend to quit more frequently than the other.

In [6]:
# Attrition by Gender
ggplot(hr_data,aes(x=Gender, fill=factor(Attrition))) +
  geom_bar() +
  ggtitle("Attrition # by Gender") +
  xlab("Gender") +
  ylab("# of Employees") +
  labs(fill="Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060")) +
  geom_text(aes(label=..count..), stat='count',
              vjust=1.5, color="white", size=3.5)

# Turns data into percentages
percentData <- hr_data %>% group_by(Gender) %>% count(Attrition) %>%
    mutate(ratio=scales::percent(n/sum(n)))

ggplot(hr_data,aes(x=Gender, fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Gender") +
  xlab("Gender") +
  ylab("% Attrition") +
  labs(fill="Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060")) +
  geom_text(data=percentData, aes(y=n,label=ratio),
              position=position_fill(vjust=0.5), color="white", size=3.5)

### QUESTION 2: Does age appear to make a difference? Are our younger workers less committed to our company?
The area chart appears to show that there are more employees who are in their late 20s and early 30s tend to leave the company. However, there are more employees in that age range, so it makes sense that more of them are leaving than other ages. The bar chart gives a better insight of attrition by age. It reveals that the younger employees below the age of 30 are leaving at a higher rate than any other age group.

In [7]:
# Attrition by Age (area chart)
ggplot(hr_data,aes(Age, fill=factor(Attrition))) +
  geom_area(stat="bin",binwidth=1) +
  ggtitle("Attrition # by Age") +
  xlab("Age") +
  ylab("# of Employees") +
  labs(fill="Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))

# Attrition Percentage by Age (bar chart)
ggplot(hr_data,aes(Age, fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Age") +
  xlab("Age") +
  ylab("% Attrition") +
  labs(fill="Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))

### QUESTION 3: Are we working our employees too hard? Is our overtime requirement for some employees driving them away?
The first bar chart shows the total number of employees who worked overtime and the number of employees that left. The second bar chart reveals that 30.5% of those who worked overtime left the company. There is a correlation between overtime and attrition. Keep in mind correlation does not mean causation.

In [8]:
# Attrition by OverTime (bar chart)
ggplot(hr_data,aes(x=OverTime, fill=factor(Attrition))) +
  geom_bar() +
  ggtitle("Attrition # by OverTime") +
  xlab("OverTime") +
  ylab("# of Employees") +
  labs(fill="Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060")) +
  geom_text(aes(label=..count..), stat='count',
              vjust=1.5, color="white", size=3.5)

# Turns data into percentages
percentData <- hr_data %>% group_by(OverTime) %>% count(Attrition) %>%
    mutate(ratio=scales::percent(n/sum(n)))

ggplot(hr_data,aes(OverTime,fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by OverTime") +
  xlab("Over Time") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060")) +
  geom_text(data=percentData, aes(y=n,label=ratio),
              position=position_fill(vjust=0.5), color="white", size=3.5)

### QUESTION 4: Is this about money? Are we not paying employees enough? Were the raises not strong enough? Is that why they leave?

Based on the chart, it appears that employees with lower incomes are leaving at higher rates. Employees with higher incomes are usually senior employees who are retiring. There is a correlation between income and attrition.

In [9]:
# Attrition % by Monthly Income (histogram)
ggplot(hr_data,aes(MonthlyIncome,fill=factor(Attrition))) +
  geom_histogram(binwidth=1000, position="fill") +
  ggtitle("Attrition % by Monthly Income") +
  xlab("Monthly Income") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))

# Distribution of Monthly Income by Attrition (boxplot)
ggplot(hr_data, aes(x=factor(Attrition), y=MonthlyIncome)) +
  geom_boxplot(aes(fill=factor(Attrition))) +
  ggtitle("Attrition by Monthly Income") +
  xlab("Attrition") +
  ylab("Monthly Income") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))

### QUESTION 5: Our concern is that the highest performing employees are leaving at a higher rate. Is that true?
A score of 3 means the employee met the objectives and a 4 means the employee exceeded most of the objectives. According to the bar chart, those who receive a score of 3 and 4 leave at about the same rate. There is no correlation between performance rating and attrition. 

In [10]:
# Turns data into percentages
percentData <- hr_data %>% group_by(PerformanceRating) %>% count(Attrition) %>%
    mutate(ratio=scales::percent(n/sum(n)))

# Attrition % by Performance Rating (bar chart)
ggplot(hr_data,aes(PerformanceRating,fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Performance Rating") +
  xlab("Performance Rating") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))+
  geom_text(data=percentData, aes(y=n,label=ratio),
              position=position_fill(vjust=0.5), color="white", size=3.5)

### ADDITIONAL GRAPHS: Attrition by Other Variables
I decided to evaluate a few other variables to find out if they affect employee attrition.

**Attrition by Marital Status:** I hypothesized that single employees are more likely to leave a company. According to the graph, 25.5% of the single employees are leaving at a higher rate than the married or divorced employees. This confirmed my hypothesis. There is a correlation between marital status and attrition.

In [11]:
# Attrition by Marital Status

# Turns data into percentages
percentData <- hr_data %>% group_by(MaritalStatus) %>% count(Attrition) %>%
    mutate(ratio=scales::percent(n/sum(n)))

ggplot(hr_data,aes(MaritalStatus,fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Marital Status") +
  xlab("Marital Status") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))+
  geom_text(data=percentData, aes(y=n,label=ratio),
              position=position_fill(vjust=0.5), color="white", size=3.5)

**Attrition by Distance From Home:** I hypothesized that employees who have a longer commute to work are likely to leave. The bar chart shows no correlation between distance from home and attrition.

In [12]:
# Attrition by Distance From Home
ggplot(hr_data,aes(DistanceFromHome,fill=factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Distance From Home") +
  xlab("Distance From Home") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))

**Attrition by Job Level:** I hypothesized that employees who are newer are more likely to leave. The bar chart shows that newer employees are leaving at a higher rate than other job level employees.

In [13]:
# Attrition by Job Level

# Turns data into percentages
percentData <- hr_data %>% group_by(JobLevel) %>% count(Attrition) %>%
    mutate(ratio=scales::percent(n/sum(n)))

ggplot(hr_data,aes(x=JobLevel, fill = factor(Attrition))) +
  geom_bar(position="fill") +
  ggtitle("Attrition % by Job Level") +
  xlab("Job Level") +
  ylab("% Attrition") +
  labs(fill = "Attrition") +
  scale_fill_manual(values=c("#B1B1B1", "#D36060"))+
  geom_text(data=percentData, aes(y=n,label=ratio),
              position=position_fill(vjust=0.5), color="white", size=3.5)

# PART 3: PREPARE THE DATASET FOR MODELING

With new insight on the data, it's time to start analyzing what affects employee attrition. First, I checked for null values in the data. Second, I split the dataset into two new datasets. One acted as my training data, where I created models for my predictions. The other dataset was my test data, where I tested my prediction models.

In [14]:
# Checks data for null values
sum(is.na(hr_data))

# If null values exists, drop records using na.omit

[1] 0

In [38]:
# Random number generator
set.seed(3456)

# Splits the data
trainIndex <- createDataPartition(hr_data$Attrition, p = .8, 
                                  list = FALSE, times = 1)

# Creates new train and test datasets
hr_train <- hr_data[trainIndex,]
hr_test  <- hr_data[-trainIndex,]

In [16]:
# Explores the first few rows of each new dataset
head(hr_train)
head(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
1,1,1,41,Female,Single,Sales,Sales Executive,5993,Yes,Life Sciences,...,3,11,0,0,8,8,6,4,0,5
2,0,2,49,Male,Married,Research & Development,Research Scientist,5130,No,Life Sciences,...,4,23,1,3,10,1,10,7,1,7
3,1,4,37,Male,Single,Research & Development,Laboratory Technician,2090,Yes,Other,...,3,15,0,3,7,6,0,0,0,0
4,0,5,33,Female,Married,Research & Development,Research Scientist,2909,Yes,Life Sciences,...,3,11,0,3,8,1,8,7,3,0
5,0,7,27,Male,Married,Research & Development,Laboratory Technician,3468,No,Medical,...,3,12,1,3,6,9,2,2,2,2
7,0,10,59,Female,Married,Research & Development,Laboratory Technician,2670,Yes,Medical,...,4,20,3,3,12,4,1,0,0,0


,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,PerformanceRating,PercentSalaryHike,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,3,13,0,2,8,0,7,7,3,6
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,4,22,1,2,1,1,1,0,0,0
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,3,13,1,5,6,0,5,4,0,3
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,3,12,2,5,7,0,6,2,0,5
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,4,23,0,4,10,7,5,3,0,3
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,3,11,0,4,13,0,12,6,2,11


# PART 4: CREATE & EVALUATE PREDICTION MODELS

Now that I have two new datasets, I created prediction models using two methods – logistic regression and classification. For the logistic regression method, I used the generalized linear model (glm) function, which compared how several variables affect different continuous variables. For the classification method, I used the random forest (rf) function, which generated many decision trees and outputted the mode of the class.

I used these models to see what affects attrition. I evaluated each model and its variable importance, so I could narrow down the variables that influence attrition. Each model progressively became more accurate at predicting attrition.

## METHOD #1: LOGISTIC REGRESSION

<b>MODEL 1:</b> My first model tested and evaluated all variables presented in the dataset to get a broad overview of which variables are important and not.

In [17]:
# Uses glm() to create a prediction model
    # Selects all variables
attrition_model_glm_1 <- glm(Attrition ~ .,
                     data = hr_train, family = binomial)

In [18]:
# Evaluates variable importance
importance <- as.data.frame(varImp(attrition_model_glm_1))
importance <- data.frame(Importance = importance$Overall,
                        Field = rownames(importance))
importance[order(-importance$Importance),]

,Importance,Field
,<dbl>,<fct>
17,8.73216057,OverTimeYes
25,4.40187743,DistanceFromHome
27,4.28942404,JobInvolvement
34,4.11241910,NumCompaniesWorked
28,3.89042105,JobSatisfaction
37,3.75136461,YearsSinceLastPromotion
23,3.59670614,BusinessTravelTravel_Frequently
36,3.01944823,YearsInCurrentRole
5,2.93624763,MaritalStatusSingle


In [19]:
# Shows variables' p-values to determine variable significance
summary(attrition_model_glm_1)


Call:
glm(formula = Attrition ~ ., family = binomial, data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.7806  -0.4957  -0.2701  -0.0983   3.5749  

Coefficients:
                                   Estimate Std. Error z value Pr(>|z|)    
(Intercept)                      -1.288e+01  4.547e+02  -0.028 0.977405    
EmployeeNumber                   -2.184e-04  1.666e-04  -1.311 0.189827    
Age                              -3.916e-02  1.489e-02  -2.629 0.008560 ** 
GenderMale                        4.978e-01  2.084e-01   2.388 0.016930 *  
MaritalStatusMarried              4.735e-01  2.996e-01   1.580 0.114028    
MaritalStatusSingle               1.127e+00  3.837e-01   2.936 0.003322 ** 
DepartmentResearch & Development  1.277e+01  4.547e+02   0.028 0.977603    
DepartmentSales                   1.258e+01  4.547e+02   0.028 0.977922    
JobRoleHuman Resources            1.301e+01  4.547e+02   0.029 0.977178    
JobRoleLaboratory Technician      1.314

<b>MODEL 2:</b> I compared the results from varImp() and summary(). Larger numbers in varImp() are important to keep. Smaller numbers that are less than 0.05 and ones with asterisks next to them in summary() are significant. From evaluating varImp() and summary(), I took out a significant number of variables from the model and tested it in model 2.

In [20]:
# Uses glm() to create a prediction model
attrition_model_glm_2 <- glm(Attrition ~ Age + MaritalStatus + JobRole +
                         OverTime + BusinessTravel + DistanceFromHome +
                         JobInvolvement + JobSatisfaction + TrainingTimesLastYear +  
                         NumCompaniesWorked + YearsInCurrentRole + YearsSinceLastPromotion +
                         YearsWithCurrManager,
                     data = hr_train, family = binomial)

In [21]:
# Evaluates variable importance
importance <- as.data.frame(varImp(attrition_model_glm_2))
importance <- data.frame(Importance = importance$Overall,
                        Field = rownames(importance))
importance[order(-importance$Importance),]

,Importance,Field
,<dbl>,<fct>
12,8.5982258,OverTimeYes
21,4.3588284,YearsSinceLastPromotion
3,4.3148820,MaritalStatusSingle
16,4.2114755,JobInvolvement
17,4.1418334,JobSatisfaction
15,4.0398987,DistanceFromHome
19,3.9143774,NumCompaniesWorked
1,3.7543265,Age
11,3.7533897,JobRoleSales Representative


In [22]:
# Shows variables' p-values to determine variable significance
summary(attrition_model_glm_2)


Call:
glm(formula = Attrition ~ Age + MaritalStatus + JobRole + OverTime + 
    BusinessTravel + DistanceFromHome + JobInvolvement + JobSatisfaction + 
    TrainingTimesLastYear + NumCompaniesWorked + YearsInCurrentRole + 
    YearsSinceLastPromotion + YearsWithCurrManager, family = binomial, 
    data = hr_train)

Deviance Residuals: 
    Min       1Q   Median       3Q      Max  
-1.6474  -0.5178  -0.2881  -0.1078   3.7575  

Coefficients:
                                Estimate Std. Error z value Pr(>|z|)    
(Intercept)                     -0.62899    0.85896  -0.732 0.464004    
Age                             -0.04793    0.01277  -3.754 0.000174 ***
MaritalStatusMarried             0.48524    0.28056   1.730 0.083704 .  
MaritalStatusSingle              1.23226    0.28558   4.315 1.60e-05 ***
JobRoleHuman Resources           0.99094    0.61989   1.599 0.109913    
JobRoleLaboratory Technician     1.41637    0.45260   3.129 0.001752 ** 
JobRoleManager                  -0.49947   

## METHOD #2: CLASSIFICATION
In addition to logistic regression modeling, I also modeled the data using classification. I created a prediction model using the random forest function to see what affects attrition. I evaluated each model and its variable importance, so I could narrow down the variables that influence attrition. Each model progressively became more accurate at predicting attrition.

**MODEL 1:** My first model tested and evaluated all variables presented in the dataset to get a broad overview of which variables are important and not.

In [23]:
# Uses randomForest() to create a prediction model
    # Selects all variables
attrition_model_rf_1 <- randomForest(Attrition ~ ., 
              data = hr_train, importance = TRUE, type = "classification") 

Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

In [25]:
# Plots variable importance
varImpPlot(attrition_model_rf_1)
options(repr.plot.width=7, repr.plot.height=5)

In [26]:
# Table view of variable importance plot
attrition_model_rf_1$importance

,%IncMSE,IncNodePurity
EmployeeNumber,-8.558727e-04,8.5279463
Age,8.782378e-03,11.3890717
Gender,6.837406e-05,1.2062078
MaritalStatus,3.042941e-03,4.5826656
Department,1.622222e-03,1.4211873
JobRole,6.822530e-03,9.2752088
MonthlyIncome,1.139995e-02,14.2926342
OverTime,1.680356e-02,10.4455537
EducationField,2.346720e-03,7.6818522
BusinessTravel,1.014646e-03,3.3252110


**MODEL 2:** I compared the results from the variable importance plot. varImpPlot() decreasingly sorts attributes under %IncMSE, which is the percent increase of mean squared error. This tells you how close the model is to the actual outcome by taking the average of the square of the errors. Variables with larger %IncMSE are important to keep. From evaluating varImpPlot(), I took out a significant number of variables from the model and tested it in model 2.

In [27]:
# Uses randomForest() to create a prediction model
attrition_model_rf_2 <- randomForest(Attrition ~ OverTime + MonthlyIncome + Age + TotalWorkingYears +
                                     JobRole + JobLevel + YearsAtCompany + MaritalStatus + 
                                     StockOptionLevel + YearsInCurrentRole + NumCompaniesWorked, 
              data = hr_train, importance = TRUE, ntree=2000, type = "classification") 

Warning message in randomForest.default(m, y, ...):
“The response has five or fewer unique values.  Are you sure you want to do regression?”

In [28]:
# Plots variable importance
varImpPlot(attrition_model_rf_2)
options(repr.plot.width=7, repr.plot.height=5)

In [29]:
# Table view of variable importance plot
attrition_model_rf_2$importance

,%IncMSE,IncNodePurity
OverTime,0.019304717,11.818579
MonthlyIncome,0.016195166,24.501117
Age,0.013164884,18.392290
TotalWorkingYears,0.013210783,13.866473
JobRole,0.009870641,13.652754
JobLevel,0.011154847,4.537877
YearsAtCompany,0.009275539,12.754004
MaritalStatus,0.006575730,6.442578
StockOptionLevel,0.006496395,6.788471
YearsInCurrentRole,0.004154849,7.836968


# PART 5: PREDICT & TEST ATTRITION USING MODEL

With my final model, I believe I have captured the most significant variables that affect attrition. I used model 2 from both methods to predict attrition in the test data. I compared my prediction model with the test data using Receiver Operating Characteristic (ROC) curves and Area Under the Curve (AUC). ROC is a probability curve and AUC represents degree or measure of separability. ROC curves show the performance of a classification model at all classification thresholds. AUC computes the area under the ROC curve with the trapezoidal rule. ROC and AUC helped me check how well my model is at predicting attrition. Higher the AUC, the better the model is at predicting 0s as 0s and 1s as 1s. In this analysis, 0 means the employee did not turnover and 1 means the employee did turnover.

### LOGISTIC REGRESSION MODEL

In [39]:
# Uses the model to predict attrition in the test data
hr_test$Attrition_predictions_glm_all <- predict(attrition_model_glm_1, 
                newdata = hr_test, type = "response")

hr_test$Attrition_predictions_glm <- predict(attrition_model_glm_2, 
                newdata = hr_test, type = "response")

In [32]:
# Evaluate the logistic regression models.

# Generate ROC curves for both version of the L.R. model.
ROC_glm_1 <- roc(hr_test$Attrition, hr_test$Attrition_predictions_glm_all)
ROC_glm_2 <- roc(hr_test$Attrition, hr_test$Attrition_predictions_glm)

# Print the AUC for both versions of the model
paste("AUC for Logistic Regression Model (all variables):", round(auc(ROC_glm_1),2), "(red)")
paste("AUC for Logistic Regression Model (limited variables):", round(auc(ROC_glm_2),2), "(blue)")

# Plot the ROC curves.
plot.roc(ROC_glm_1, col="red")
lines.roc(ROC_glm_2, col="blue")

Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases


[1] "AUC for Logistic Regression Model (all variables): 0.8 (red)"

[1] "AUC for Logistic Regression Model (limited variables): 0.81 (blue)"

In [33]:
# View first few records to make sure predictions have been added.
head(hr_test)
str(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,StockOptionLevel,TrainingTimesLastYear,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_predictions_glm_all,Attrition_predictions_glm
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,0,2,8,0,7,7,3,6,0.12131264,0.096898529
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,1,2,1,1,1,0,0,0,0.21684701,0.151068850
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,1,5,6,0,5,4,0,3,0.03863858,0.028403024
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,2,5,7,0,6,2,0,5,0.03694925,0.031333557
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,0,4,10,7,5,3,0,3,0.64300933,0.555198995
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,0,4,13,0,12,6,2,11,0.00141107,0.001617293


'data.frame':	294 obs. of  27 variables:
 $ Attrition                    : int  0 0 0 0 1 0 0 1 0 0 ...
 $ EmployeeNumber               : int  8 11 14 21 27 28 30 31 36 52 ...
 $ Age                          : int  32 30 35 32 36 34 21 34 44 33 ...
 $ Gender                       : Factor w/ 2 levels "Female","Male": 2 2 2 2 2 1 2 2 1 1 ...
 $ MaritalStatus                : Factor w/ 3 levels "Divorced","Married",..: 3 1 2 1 3 3 3 3 2 2 ...
 $ Department                   : Factor w/ 3 levels "Human Resources",..: 2 2 2 2 3 2 2 2 2 3 ...
 $ JobRole                      : Factor w/ 9 levels "Healthcare Representative",..: 3 3 3 7 9 6 7 7 1 8 ...
 $ MonthlyIncome                : int  3068 2693 2426 3298 3407 11994 1232 2960 10248 5376 ...
 $ OverTime                     : Factor w/ 2 levels "No","Yes": 1 1 1 2 1 1 1 1 1 1 ...
 $ EducationField               : Factor w/ 6 levels "Human Resources",..: 2 2 4 2 2 2 2 4 4 2 ...
 $ BusinessTravel               : Factor w/ 3 levels "Non-Travel

### CLASSIFICATION MODEL

In [34]:
# Uses the model to predict attrition in the test data
hr_test$Attrition_predictions_rf_all <- predict(attrition_model_rf_1, 
                newdata = hr_test, type = "response")

hr_test$Attrition_predictions_rf <- predict(attrition_model_rf_2, 
                newdata = hr_test, type = "response")

In [35]:
# Evaluate the classification models.

# Generate ROC curves for both version of the classification model.
ROC_rf_1 <- roc(hr_test$Attrition, hr_test$Attrition_predictions_rf_all)
ROC_rf_2 <- roc(hr_test$Attrition, hr_test$Attrition_predictions_rf)

# Print the AUC for both versions of the model
paste("AUC for Classification Model (all variables):", round(auc(ROC_rf_1),2), "(red)")
paste("AUC for Classification Model (limited variables):", round(auc(ROC_rf_2),2), "(blue)")

# Plot the ROC curves.
plot.roc(ROC_rf_1, col="red")
lines.roc(ROC_rf_2, col="blue")

Setting levels: control = 0, case = 1
Setting direction: controls < cases
Setting levels: control = 0, case = 1
Setting direction: controls < cases


[1] "AUC for Classification Model (all variables): 0.76 (red)"

[1] "AUC for Classification Model (limited variables): 0.73 (blue)"

In [36]:
# View first few records to make sure predictions have been added.
head(hr_test)
str(hr_test)

,Attrition,EmployeeNumber,Age,Gender,MaritalStatus,Department,JobRole,MonthlyIncome,OverTime,EducationField,...,TotalWorkingYears,NumCompaniesWorked,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager,Attrition_predictions_glm_all,Attrition_predictions_glm,Attrition_predictions_rf_all,Attrition_predictions_rf
,<int>,<int>,<int>,<fct>,<fct>,<fct>,<fct>,<int>,<fct>,<fct>,...,<int>,<int>,<int>,<int>,<int>,<int>,<dbl>,<dbl>,<dbl>,<dbl>
6,0,8,32,Male,Single,Research & Development,Laboratory Technician,3068,No,Life Sciences,...,8,0,7,7,3,6,0.12131264,0.096898529,0.06420000,0.11369438
8,0,11,30,Male,Divorced,Research & Development,Laboratory Technician,2693,No,Life Sciences,...,1,1,1,0,0,0,0.21684701,0.151068850,0.39470000,0.27477970
11,0,14,35,Male,Married,Research & Development,Laboratory Technician,2426,No,Medical,...,6,0,5,4,0,3,0.03863858,0.028403024,0.12256667,0.04902305
17,0,21,32,Male,Divorced,Research & Development,Research Scientist,3298,Yes,Life Sciences,...,7,0,6,2,0,5,0.03694925,0.031333557,0.18773333,0.14008274
22,1,27,36,Male,Single,Sales,Sales Representative,3407,No,Life Sciences,...,10,7,5,3,0,3,0.64300933,0.555198995,0.26853333,0.09132118
23,0,28,34,Female,Single,Research & Development,Research Director,11994,No,Life Sciences,...,13,0,12,6,2,11,0.00141107,0.001617293,0.02403333,0.03085378


'data.frame':	294 obs. of  29 variables:
 $ Attrition                    : int  0 0 0 0 1 0 0 1 0 0 ...
 $ EmployeeNumber               : int  8 11 14 21 27 28 30 31 36 52 ...
 $ Age                          : int  32 30 35 32 36 34 21 34 44 33 ...
 $ Gender                       : Factor w/ 2 levels "Female","Male": 2 2 2 2 2 1 2 2 1 1 ...
 $ MaritalStatus                : Factor w/ 3 levels "Divorced","Married",..: 3 1 2 1 3 3 3 3 2 2 ...
 $ Department                   : Factor w/ 3 levels "Human Resources",..: 2 2 2 2 3 2 2 2 2 3 ...
 $ JobRole                      : Factor w/ 9 levels "Healthcare Representative",..: 3 3 3 7 9 6 7 7 1 8 ...
 $ MonthlyIncome                : int  3068 2693 2426 3298 3407 11994 1232 2960 10248 5376 ...
 $ OverTime                     : Factor w/ 2 levels "No","Yes": 1 1 1 2 1 1 1 1 1 1 ...
 $ EducationField               : Factor w/ 6 levels "Human Resources",..: 2 2 4 2 2 2 2 4 4 2 ...
 $ BusinessTravel               : Factor w/ 3 levels "Non-Travel

# PART 6: SAVE PREDICTIONS FILE & RECOMMENDATIONS
I decided to use my logistics regression model for predicting because its AUC value was 0.812, which was higher that the classification model. This means that there is an 81.2% chance that the logistics regression model will be able to distinguish between whether or not the employee left. I saved the predictions in a new CSV file, using write.csv(). Lastly, I recommended suggestions for retaining employees longer based on the significant variables I found to have been a factor in employees leaving the company.

In [40]:
# Writes predictions (in the test data) to a new CSV file
write.csv (hr_test[,c(2,1,27)], "attrition_predictions.csv", row.names = FALSE)

## RECOMMENDATIONS
In Soundarapandiyan's research, it states some strategies for reducing employee turnover, such as "accommodation of individual preferences on working hours, regular appraisals, providing as much job security." 

Based off my analysis of employee attrition, I recommended suggestions for four variables the company could focus on to retain more employees in the company.

**Age & Marital Status:**
From my analysis, I believe that the company can minimize employee turnover by focusing on those who are single and under 30 years old. Usually, younger employees are likely to turnover because they view the job opportunity as a gap filler occupation while thinking of other career alternatives. I suggest making it more attractive to stay at the company by offering employee benefits and adding incentives for reaching a certain goal. Employers should be aware of the benefits that other companies offer so they could compete with the industry (Ahuja, 2013). Soundarapandiyan suggests providing clear career paths, communicating with employees, giving support for personal and career development, and considering work-life balance issues.

**Over Time:**
The company could offer compensatory time for those who work overtime. However, this suggestion needs more information about how much overtime these employees are working. The manager will need to weigh out the costs and benefits of paying for overtime or hiring new/temporary employees to reduce the workload.

**Job Satisfaction:**
Studies have shown that job satisfaction is "one of the major predictor of turnover" (Soundarapandiyan, 2015). If employees are not happy with their job, then they are likely to leave. Ahuja says, "Employees prefer to work in an environment which is suitable for them." I recommend creating a pleasant working environment. I also suggest that managers regularly meet with their employees or allow employees to voice their opinions on changes and other issues. Also, ensuring that employees have tasks that demand their full potential and gain relevant work experiences will keep employees satisfied (Ahuja, 2013).

<span style=text-decoration:underline>Works Cited</span>

Ahuja, Jaya, and Vani Narula. “Causes Of Employee Attrition In BPO Companies Based In Delhi NCR Region.” Ethics & Critical Thinking Journal, vol. 2013, no. 3, Sept. 2013, pp. 15–87. EBSCOhost, search.ebscohost.com/login.aspx?direct=true&db=ent&AN=93342081&site=eds-live.

Soundarapandiyan, K., and M. Ganesh. “Employee Retention Strategy with Reference to Chennai Based Ites Industry- an Empirical Study.” Global Management Review, vol. 9, no. 2, Feb. 2015, pp. 1–13. EBSCOhost, search.ebscohost.com/login.aspx?direct=true&db=bth&AN=111502517&site=eds-live.